In [ ]:
# Hack to make the module importable
import sys
sys.path.append(r'./../')

In [ ]:
%load_ext autoreload
%autoreload 2
import neo4j
import pandas as pd

from rel2graph.relational_modules.pandas import PandasDataFrameIterator
from rel2graph import IteratorIterator
from rel2graph import Converter
from rel2graph.utils import load_file
from rel2graph import register_attribute_postprocessor, Attribute

import rel2graph.common_modules.types # For FLOAT, INT, etc. wrappers
# This is required because the pandas dataframe iterator will convert all values 
# to int64 which is not supported by neo4j

In [ ]:
# Configure Logging
import logging

#logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger("rel2graph")
logger.setLevel(logging.INFO)
log_formatter = logging.Formatter("%(asctime)s [%(threadName)s]::[%(levelname)s]::%(filename)s: %(message)s")
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)


In [ ]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris

In [ ]:
data = {
  "ID": [1,2,2,3,4,4],
  "FirstName": ["Julian", "Fritz",  "Fritz", "Hans", "Rudolfo", "Rudolfo"],
  "LastName": ["Minder", "Generic", "SomeGuy", "Müller", "Muster", "Muster"],
  "FavoriteFlower": ["virginica", "setosa", "setosa", "versicolor", "setosa", "setosa"]
}
people = pd.DataFrame(data)
people

In [ ]:
schema = """
ENTITY("Flower"):
    NODE("Flower") flower:
        - sepal_length = FLOAT(Flower.sepal_length)
        - petal_length = FLOAT(Flower.petal_width)
        - sepal_width = FLOAT(Flower.sepal_width)
        - petal_width = FLOAT(Flower.petal_width)
    NODE("Species", "BioEntity") species:
        + Name = Flower.species
    RELATIONSHIP(flower, "is", species):
    
ENTITY("Person"):
    NODE("Person") person:
        + ID = INT(Person.ID)
        - FirstName = Person.FirstName
        - LastName = Person.LastName

    RELATIONSHIP(person, "likes", MATCH("Species", Name=Person.FavoriteFlower)):
        - Since = "4ever"
"""

In [ ]:
uri = "bolt://localhost:7687"
auth = neo4j.basic_auth("neo4j", "password") # CHANGE TO YOUR CREDENTIALS

In [ ]:
# Delete all nodes and relationships
driver = neo4j.GraphDatabase().driver(uri, auth=auth)
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

In [ ]:
iterator = IteratorIterator([PandasDataFrameIterator(people, "Person"), PandasDataFrameIterator(iris, "Flower")])

In [ ]:
converter = Converter(schema, iterator, uri, auth, num_workers=1)

In [ ]:
from tqdm.notebook import tqdm
converter(progress_bar=tqdm)